In [ ]:
# initialize accelerator and trackers (if enabled)
import torchvision.transforms as T
import os
import numpy as np
import matplotlib.pyplot as plt
from os import makedirs, path, scandir
import pickle
import cv2
import json
from yacs.config import CfgNode as CN
from torch.utils.data import Dataset
from tqdm import tqdm
import logging
import random
import torch
# from src.datasets import FFPP,RPPG
from accelerate import Accelerator
from main import get_config, init_accelerator, set_seed, FFPP, DFDC, CDF
logging.basicConfig(level="DEBUG", format='[%(levelname)s][%(filename)s:%(lineno)d]: %(message)s')


class Obj:
    pass


def generate_samples(
    sample_num,
    num_frames=1,
    duration=5,
    stride=1,
    path="./misc/extern/test/ffpp_sample/",
    splits=["train", "val", "test"]
):
    plt.ioff()
    transform = T.Compose([
        T.Resize(224, interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(224),
        T.ConvertImageDtype(torch.float32),
    ])
    for split in splits:
        c = FFPP.get_default_config()
        c.augmentation = "normal+rrc"
        c.contrast = 1
        c.compressions = ["c23"]
        c.types = ["REAL", "DF", "FS", "F2F", "NT"]
        c.root_dir = "./datasets/ffpp/"
        c.vid_ext = ".avi"
        accelerator = Accelerator(mixed_precision='no')
        x = FFPP(c, num_frames, duration, transform, accelerator, split=split)
        folder = os.path.join(path, split)
        os.makedirs(folder, exist_ok=True)

        for i in range(sample_num):
            frames, label, mask, _, _ = x[random.randint(0, len(x))]
            plt.figure(figsize=(2 * (num_frames // stride), 4), layout="constrained")
            plt.suptitle(f"label={label}")
            plt.subplot(2, 1, 1)
            plt.title(f"mask={mask[0]}")
            print(frames[0]["c23"].shape)
            plt.imshow(
                np.stack(
                    frames[0]["c23"][::stride].numpy().transpose((0, 2, 3, 1)), axis=1
                ).reshape((x.n_px, -1, 3))
            )
            plt.subplot(2, 1, 2)
            plt.title(f"mask={mask[1]}")
            plt.imshow(
                np.stack(
                    frames[1]["c23"][::stride].numpy().transpose((0, 2, 3, 1)), axis=1
                ).reshape((x.n_px, -1, 3))
            )
            # plt.show()
            plt.savefig(os.path.join(folder, f"{i}.jpg"))
            plt.close()
            # return

In [ ]:
generate_samples(200, splits=["train"])

In [ ]:
c = FFPP.get_default_config()
c.augmentation = "normal+rrc"
# c.pair = 1
c.contrast = 1
# c.compressions = ["c23", "raw"]
# c.compressions = ["raw", "c23"]
c.compressions = ["c23"]
# c.ssl_fake = 1
c.types = ["REAL", "DF", "FS", "F2F", "NT"]
# c.types = ["REAL"]
c.root_dir = "./datasets/ffpp/"
c.vid_ext = ".avi"
# c.types = ["DF", "FS", "F2F"]
accelerator = Accelerator(mixed_precision='no')
x = FFPP(c, 1, 5, lambda x: x, accelerator, split="train")
c

In [ ]:
# x.video_table["REAL"]["c23"]

In [ ]:
# typ, cmp, idx, _ = x.video_list[1234]
# x.video_table[typ][cmp][idx]

In [ ]:
# a = x.get_dict(7079, True)

In [ ]:
len(x)

In [ ]:
frames, label, mask, _, _ = x[random.randint(0, len(x))]
# frames, label, mask, _, _ = x[9751]
(len(frames), label, len(mask))

In [ ]:
frames[0].keys(), frames[1].keys()

In [ ]:

plt.figure(figsize=(50, 5))
plt.subplot(2, 1, 1)
plt.imshow(
    np.stack(
        frames[0]["c23"][:30].numpy().transpose((0, 2, 3, 1)), axis=1
    ).reshape((x.n_px, -1, 3))
)
plt.subplot(2, 1, 2)
plt.imshow(
    np.stack(
        frames[1]["c23"][:30].numpy().transpose((0, 2, 3, 1)), axis=1
    ).reshape((x.n_px, -1, 3))
)

In [ ]:
# for i in tqdm(range(len(x))):
#     try:
#         x[i]
#     except Exception as e:
#         print(f"Error Occur at {i}:{e}")